In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
import torch
torch.set_float32_matmul_precision('medium')

In [3]:
from datasets.shapenet.shapenet_loader import get_dataloaders

categories = ['chair', 'airplane']
path = "../data/ShapeNet"
tr, te, val = get_dataloaders(path, categories=categories, load_renders=True)

Loading (val) renders for chair (03001627): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 662/662 [00:18<00:00, 36.54it/s]


In [4]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 256],
    "num_layers": 1,
    "attn_heads": (None, None, 8, 8, 8),
    "cross_attn_heads": (None, 8, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192, 128],
        "num_layers": 1,
        "attn_heads": 8,
        "cross_attn_heads": 8
    },
    {
        "features": [128, 64, 32],
        "num_layers": 1,
        "attn_heads": None,
        "cross_attn_heads": (8, None)
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

from my_models.lightning_base import DiffusionBase
model = DiffusionBase(model, lr=1e-4)
model.set_noise_scheduler(tr.dataset.noise_scheduler)

In [8]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint

# Save every 50 epochs, with epoch number in filename
checkpoint = ModelCheckpoint(
    dirpath="../checkpoints/ShapeNet",
    filename="chair-airplane-{epoch:03d}",  # e.g., model-epoch=050.ckpt
    every_n_epochs=50,            # Save interval
    save_top_k=-1,                # Keep all checkpoints (-1 = no limit)
)

epochs = 2000
trainer = L.Trainer(
    log_every_n_steps=5,
    max_epochs=epochs,
    callbacks=[checkpoint],
    gradient_clip_val=10.0,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=val)

/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ubuntu/SPVD_Lightning/checkpoints/ShapeNet exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 36.2 M | train
------------------------------------------
36.2 M    Trainable params
0         Non-trainable params
36.2 M    Total params
144.709   Total estimated model params size (MB)
408       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7684580a9d80>
Traceback (most recent call last):
  File "/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1568, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/envs/spvd/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/opt/conda/envs/spvd/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/conda/envs/spvd/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/opt/conda/envs/spvd/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 127620, 127624, 127628, 127632) exited unexpectedly

In [ ]:
import os
os.makedirs("../checkpoints/ShapeNet/", exist_ok=True)
checkpoint = {
    "state_dict": model.state_dict(),
    "down_blocks": down_blocks,
    "up_blocks": up_blocks,
}
torch.save(checkpoint, "../checkpoints/ShapeNet/chair-airplane-2000.ckpt")

In [10]:
import numpy as np
samples = 16
references = [tr.dataset[idx] for idx in np.random.choice(list(range(len(tr.dataset))), size=(16,))]
reference_images = torch.stack([r["render-features"] for r in references]).to("cuda")

In [11]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
ddpm_sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=1024)

In [12]:
samples = 16
model = model.cuda().eval()
preds = ddpm_sched.sample(model, samples, 4096, reference=reference_images, stochastic=True)


Sampling: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [03:02<00:00,  5.62it/s]


In [13]:
from utils.visualization import display_pointclouds_grid

In [14]:
display_pointclouds_grid(preds.cpu().numpy(), offset=8, point_size=0.3)

Output()

In [15]:
real = np.array([r["pc"] for r in references])
display_pointclouds_grid(real, offset=8, point_size=0.3)

Output()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig, axs = plt.subplots(4, 4, figsize=(10, 10))

for i in range(4):
    for j in range(4):
        ax = axs[i][j]
        ax.axis('off')
        idx = i * 4 + j
        view = references[idx]["selected-view"]
        file = references[idx]["filename"]

        img = mpimg.imread(f"../data/ShapeNet/renders/{file}/00{view}.png")
        ax.imshow(img)